# Figure 6 - Analysis

In [ ]:
import sys
sys.path.append("..")

from main import *
from zebrafish import *

from scipy.stats import zscore, spearmanr
from scipy.stats import percentileofscore
from brainsmash.mapgen.base import Base

# Setting paths

In [ ]:
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'

atlas = Mapzebrain(atlas_path)
mask_tectum = atlas.get_region_mask(22)
mask_tectum[:, :, :284] = 0

In [ ]:
top_directory = '/media/anleg84/KINGSTON/Datasets/Geometry/'
try:
    datasets = identify_folders(top_directory, ['920nm'])
except:
    datasets = None

# Validating which tectum nodes contain cells

Anatomical nodes within the optic tectum were computed beforehand in the `Figure6-Meshes.ipynb` notebook. Here we verify which anatomical positions within the tectum were appropriately sampled by the multiplane imaging across animals. Any nodes with poor sampling (that is, no imaged cells in at least 50% of animals) are excluded.

In [ ]:
compute = True
nodes_tectum = np.load('../Files/nodes_tectum_right_400.npy') * 2

if compute:

    N_in_tectum = []
    centroids_tectum = []
    
    for folder in tqdm(datasets):
        
        data = load_data(folder)
        centroids = data['centroids_atlas']
        in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
        
        N_in_tectum.append(np.sum(in_tectum))
        centroids_tectum.append(centroids[in_tectum])
    
    vectors = []
    for c in centroids_tectum:
        nn, _ = find_nearest_neighbors(centroids_tectum[0], nodes_tectum)
        vector = np.zeros((nodes_tectum.shape[0], ))
        vector[np.unique(nn)] = 1
        vectors.append(vector)
    vectors = np.stack(vectors, axis=0)
    
    sampled = (np.sum(vectors > 0, axis=0) >= 0.5 * len(datasets)) # At least 1 neuron in 50% of animals
    
    np.save('../Files/nodes_tectum_right_sampled.npy', nodes_tectum[sampled])

# Computing tectal FC

In [ ]:
mask_tectum = atlas.get_region_mask(22)
mask_tectum[:, :, :284] = 0 # Focusing on right hemisphere (x=284 is the saggital midplane)

nodes_tectum = np.load('../Files/nodes_tectum_right_sampled.npy')
print(nodes_tectum.shape)

In [ ]:
N_neurons_in_tectum = []
N_neurons_per_node = []

Here we load the time series from each animal, detrend and temporally filter them, map the neurons to tectal nodes, and finally compute pairwise correlations of node time series.

In [ ]:
FC_matrices = []

for i, folder in tqdm(enumerate(datasets)):
    
    data = load_data(folder)
    
    centroids = data['centroids_atlas']
    
    in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
    centroids_tectum = centroids[in_tectum]
    dff_tectum = data['timeseries'][in_tectum]
    dff_tectum = compute_dff_using_minfilter(dff_tectum, window=120, sigma1=3, sigma2=60)
    dff_tectum = filter_timeseries(dff_tectum, 2) # 1 second gaussian filter to reduce noise and improve correlations
    N_neurons_in_tectum.append(dff_tectum.shape[0])

    # Mapping neurons to tectal nodes
    nn, _ = find_nearest_neighbors(centroids_tectum, nodes_tectum)
    node_timeseries = np.zeros((nodes_tectum.shape[0], dff_tectum.shape[1]))
    for i in range(nodes_tectum.shape[0]):
        neurons_in_node = (nn == i) # Neurons whose nearest neighbor corresponds to the tectum node i
        N_neurons_per_node.append(np.sum(neurons_in_node))
        if np.any(neurons_in_node):
            if np.sum(neurons_in_node) == 1:
                node_timeseries[i] = dff_tectum[neurons_in_node]
            else:
                node_timeseries[i] = np.mean(dff_tectum[neurons_in_node], axis=0)

    # Computing correlations
    FC = np.corrcoef(node_timeseries)
    FC_matrices.append(FC)

FC_matrices = np.stack(FC_matrices, axis=0)

Printing the number of neurons in the right tectal hemisphere:

In [ ]:
print(np.mean(N_neurons_in_tectum), '+/-', np.std(N_neurons_in_tectum))

Printing the number of neurons per tectum node:

In [ ]:
print(np.mean(N_neurons_per_node), '+/-', np.std(N_neurons_per_node))

# FC vs distance

Computing the correlation-distance relationship from the average FC matrix.

In [ ]:
FC = np.nanmean(FC_matrices[:], axis=0)
#FC = FC_matrices[9]
FC[np.diag_indices(FC.shape[0])] = 0
triangle = np.triu_indices(FC.shape[0], 1)

In [ ]:
d = compute_distances(nodes_tectum, nodes_tectum)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
ax.scatter(d[triangle], FC[triangle], alpha=0.1, color='black', edgecolor=None, s=1, rasterized=True)
ax.set_xlabel('Distance (microns)')
ax.set_ylabel('Correlation')
ax.spines[['top', 'right']].set_visible(False)

In [ ]:
np.save('../Results/figure6_scatter_distance.npy', np.stack([d[triangle], FC[triangle]], axis=0))

In [ ]:
spearmanr(d[triangle], FC[triangle])[0]

# Tectal correlations

In [ ]:
order = np.argsort(nodes_tectum[:, 1])

In [ ]:
plt.figure(figsize=(5, 5), dpi=150)
plt.imshow(FC[order, :][:, order], vmin=-0.25, vmax=0.25, cmap='coolwarm')
plt.xlabel('Node $j$')
plt.ylabel('Node $i$')

In [ ]:
np.save('../Results/figure6_tectum_correlations.npy', FC[order, :][:, order])

Plotting tectal nodes on brain atlas.

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(atlas.XYprojection, cmap='gray')
plt.scatter(nodes_tectum[:, 0], nodes_tectum[:, 1], s=5, c=nodes_tectum[:,1], edgecolor=None, cmap='rainbow')
plt.axis('off')

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(atlas.YZprojection, cmap='gray')
plt.scatter(359 - nodes_tectum[:, 2], nodes_tectum[:, 1], s=5, c=nodes_tectum[:,1], edgecolor=None, cmap='rainbow')
plt.axis('off')

# Computing eigenmode-gradient correlations

In [ ]:
N_modes = FC.shape[0]
gradients, _ = diffusion_mapping(np.abs(FC), n_components=N_modes)

vertices = np.load('Files/tectum_vertices_right.npy') * 40
vertices = np.stack([vertices[:, 2], vertices[:, 1], vertices[:, 0]], axis=1)
eigenmodes = np.load('Files/tectum_eigenmodes_right.npy')[1:]

In [ ]:
d = compute_distances(vertices, nodes_tectum)
eigenmodes = eigenmodes[:, np.argmin(d, axis=0)]

In [ ]:
mode_similarity, mapping = compute_mode_similarity_matrix(eigenmodes[:30], gradients.T[:30], return_mapping=True)

In [ ]:
np.mean(np.abs(np.diag(mode_similarity))[:10])

In [ ]:
np.save('../Results/figure6_similarity_matrix.npy', mode_similarity)
np.save('../Results/figure6_mode_mapping.npy', mapping)

In [ ]:
plt.figure(figsize=(5, 5), dpi=150)

plt.imshow(np.abs(mode_similarity), cmap='Reds', vmin=0.1, vmax=1)
plt.ylabel('Geometric modes')
plt.xlabel('Functional gradients')

#### Saving an extended version of the mode similarity matrix

In [ ]:
mode_similarity_50, mapping_50 = compute_mode_similarity_matrix(eigenmodes[:50], gradients.T[:50], return_mapping=True)

In [ ]:
np.save('../Results/figure6_similarity_matrix_50.npy', mode_similarity_50)

# Saving eigenmode and gradient figures

Pre-rendering 3D gradients and eigenmodes in .png format to embed in the multipanel figure later.

In [ ]:
figures = []

for i in range(30):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(2, 2), dpi=300)
    ax.scatter(nodes_tectum[:, 0], nodes_tectum[:, 1], nodes_tectum[:, 2], c=np.sign(mode_similarity[i, i]) * gradients[:, mapping[i]], alpha=0.75, cmap='coolwarm', s=25, edgecolor='None')
    ax.set_axis_off()
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_zlim(ax.get_zlim()[::-1])
    ax.view_init(elev=30, azim=55)
    #plt.show()

    fig_array = figure_to_array(fig)[120:440, 120:500, :]
    figures.append(fig_array)
    plt.close()

In [ ]:
plt.imshow(figures[0])

In [ ]:
np.save('../Results/figure6_tectal_gradients.npy', np.stack(figures, axis=0))

In [ ]:
vertices = np.load('../Files/tectum_vertices_right.npy') * 40
vertices = np.stack([vertices[:, 2], vertices[:, 1], vertices[:, 0]], axis=1)
eigenmodes = np.load('../Files/tectum_eigenmodes_right.npy')[1:]

In [ ]:
figures = []

for i in range(30):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(2, 2), dpi=300)
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c=eigenmodes[i], alpha=0.75, cmap='coolwarm', s=25, edgecolor='None')
    ax.set_axis_off()
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_zlim(ax.get_zlim()[::-1])
    ax.view_init(elev=30, azim=55)
    #plt.show()

    fig_array = figure_to_array(fig)[120:440, 120:500, :]
    figures.append(fig_array)
    plt.close()

In [ ]:
plt.imshow(figures[0])

In [ ]:
np.save('../Results/figure6_tectal_eigenmodes.npy', np.stack(figures, axis=0))

# Null model for spatial statistics

In [ ]:
def shuffle_vector_brainsmash(vector, distances, resample=True, n_iters=1):
    base = Base(x=vector, D=distances, resample=resample)
    surrogate = base(n=n_iters)
    return surrogate

def shuffle_vectors_brainsmash(vectors, distances, resample=True):
    shuffled_vectors = []
    for v in vectors:
        base = Base(x=v, D=distances, resample=resample)
        surrogate = base(n=1)
        shuffled_vectors.append(surrogate)
    return np.array(shuffled_vectors)

In [ ]:
empirical_score = np.mean(np.abs(np.diag(mode_similarity))[:30])
print(empirical_score)

In [ ]:
d = compute_distances(nodes_tectum, nodes_tectum)
modes1, modes2 = eigenmodes[:30], gradients.T[:30]

In [ ]:
null_similarity_matrices = []

for _ in tqdm(range(1000)):
    
    shuffled_modes1 = shuffle_vectors_brainsmash(modes1, d, resample=False)
    corrs_null, mapping = compute_mode_similarity_matrix(shuffled_modes1, modes2, return_mapping=True)
    null_similarity_matrices.append(corrs_null)

null_similarity_matrices = np.stack(null_similarity_matrices, axis=0)

In [ ]:
np.save('../Results/null_similarity_scores_tectum.npy', null_similarity_matrices)

In [ ]:
null_scores = []
for m in null_similarity_matrices:
    null_scores.append(np.mean(np.abs(np.diag(m))[:30]))

In [ ]:
print(percentileofscore(null_scores, empirical_score))

Here, we obtain that the empirical score (average eigenmode-gradient correlation) is in the 100th percentile over 1000 permutations, thus $P<0.001$.

In [ ]:
plt.imshow(np.abs(null_similarity_matrices[0]), cmap='Reds', vmin=0.1, vmax=0.75)
plt.ylabel('Geometric modes')
plt.xlabel('Functional gradients')

# Extracting neuronal time series from one animal to save for figure

Uses [Rastermap](https://github.com/MouseLand/rastermap) to sort the neurons and make a nice looking plot.

In [ ]:
from rastermap import Rastermap

def compute_rastermap(timeseries, locality=0.5, time_lag_window=5, return_embedding=False):
    model = Rastermap(locality=locality, time_lag_window=time_lag_window).fit(timeseries)
    if return_embedding:
        return model.embedding
    else:
        return model.isort

##### Example planes

In [ ]:
data = load_data(datasets[0])
planes = data['average_frames']
np.save('../Results/figure6_imaging_planes.npy', planes)

##### Example time series

In [ ]:
# Example time series
data = load_data(datasets[-3])
centroids = data['centroids_atlas']
in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
centroids_tectum = centroids[in_tectum]
dff_tectum = data['timeseries'][in_tectum]
dff_tectum = compute_dff_using_minfilter(dff_tectum, window=60, sigma1=1, sigma2=30)
dff_tectum = filter_timeseries(dff_tectum, 2)
order = compute_rastermap(dff_tectum)
dff_tectum = dff_tectum[order]

np.save('../Results/figure6_timeseries.npy', dff_tectum)